In [1]:
import pandas as pd

In [2]:
# Loading CSV's
claims = pd.read_csv('dataset/synthetic_claims.csv')
patients = pd.read_csv('dataset/synthetic_patients.csv')
procedures = pd.read_csv('dataset/synthetic_procedures.csv')
providers = pd.read_csv('dataset/synthetic_providers.csv')


In [3]:
df = claims \
    .merge(patients, on='patient_id', how='left') \
    .merge(providers, on='provider_id', how='left') \
    .merge(procedures, on='procedure_code', how='left')

In [4]:
# Creating fraud flags
df['is_outlier'] = df['claim_amount'] > 9555.79
df['is_repeated_147'] = df['claim_amount'] == 147.95
df['missing_diag'] = df['diagnosis_code'].isnull() | (df['diagnosis_code'].str.strip() == '')
df['is_suspicious'] = df[['is_outlier', 'is_repeated_147', 'missing_diag']].any(axis=1)

In [5]:
# Saving as csv
df.to_csv('fraud_dataset.csv', index=False)

## Validating SQL Findings with Python (Pandas + Matplotlib)

To ensure the accuracy and reproducibility of my SQL-based fraud detection analysis, I re-implemented the core logic in Python using Pandas. This enabled me to confirm that suspicious billing patterns, high-risk providers, and missing data indicators were consistently detected across platforms.

All findings from the SQL scripts (e.g., `claims_fraud_detection.sql`) were programmatically validated using the same thresholds and grouping logic.

In [6]:
# 1. Suspicious vs Legit Claims (You may have an 'is_suspicious' flag added during processing)
suspicious_summary = df['is_suspicious'].value_counts().rename(index={True: 'Suspicious', False: 'Legit'})
print("🔹 Suspicious vs Legit Claims:\n", suspicious_summary)


🔹 Suspicious vs Legit Claims:
 is_suspicious
Suspicious    443
Legit          57
Name: count, dtype: int64


In [7]:
# 2. Repeated $147.95 Claim Pattern (SQL: HAVING frequency > 50)
repeated_147_count = df[df['claim_amount'] == 147.95].shape[0]
print("\n🔹 Repeated $147.95 Claims Count:", repeated_147_count)



🔹 Repeated $147.95 Claims Count: 419


In [8]:
# 3. Average Claim Amount by Diagnosis (SQL: GROUP BY diagnosis_code)
avg_claim_by_diag = df.groupby('diagnosis_code')['claim_amount'].agg(['count', 'mean', 'max']).sort_values(by='mean', ascending=False)
print("\n🔹 Top Diagnosis Codes by Avg Claim:\n", avg_claim_by_diag.head())



🔹 Top Diagnosis Codes by Avg Claim:
                 count         mean       max
diagnosis_code                              
D001               74  2545.818649  11532.04
D002               86  1701.112791  11698.86
D004              114  1328.557193  11532.07
D003               39   990.900256   9683.64
D005              187   474.463743   8156.92


In [9]:
# 4. High-Value Outlier Claims (SQL: claim_amount > 8284.97)
outlier_claims = df[df['claim_amount'] > 8284.97]
print("\n🔹 Outlier Claims (> $8284.97):\n", outlier_claims[['claim_id', 'claim_amount']].head())


🔹 Outlier Claims (> $8284.97):
     claim_id  claim_amount
117     C007       9683.64
118     C010       9683.93
119     C008       9683.41
120     C009       9683.57
144     C034      11209.51


In [10]:
# 5. Missing or Blank Diagnosis (SQL: WHERE diagnosis_code IS NULL OR TRIM(diagnosis_code) = '')
missing_diag_claims = df[df['diagnosis_code'].isnull() | (df['diagnosis_code'].str.strip() == '')]
print("\n🔹 Missing Diagnosis Claims:\n", missing_diag_claims[['claim_id', 'diagnosis_code']])



🔹 Missing Diagnosis Claims:
 Empty DataFrame
Columns: [claim_id, diagnosis_code]
Index: []


In [11]:
# 6. Providers with High Avg Claim Value (SQL: HAVING AVG(claim_amount) > 10000)
provider_avg_claims = df.groupby('provider_id')['claim_amount'].mean()
flagged_providers = provider_avg_claims[provider_avg_claims > 10000].index.tolist()
claims_from_flagged_providers = df[(df['provider_id'].isin(flagged_providers)) & (df['claim_amount'] > 8284.97)]
print("\n🔹 Claims from Flagged Providers:\n", claims_from_flagged_providers[['claim_id', 'provider_id', 'claim_amount']].head())


🔹 Claims from Flagged Providers:
 Empty DataFrame
Columns: [claim_id, provider_id, claim_amount]
Index: []
